In [24]:
# !pip install scikit-learn datasets

In [25]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [26]:
fib_ds = load_dataset('r-three/fib', split='test')
fib_df = fib_ds.to_pandas()

Repo card metadata block was not found. Setting CardData to empty.


In [27]:
# Visualize the CNN/DM data
fib_df.loc[fib_df['dataset'] == 'cnn_dm', ['input', 'list_choices', 'correct_choice']].head(5)

,input,list_choices,correct_choice
3122,( cnn ) the american pharmacists association i...,[<t> the american pharmacists association pass...,<t> the american pharmacists association passe...
3123,( cnn ) oprah 's in there . so 's bill murray ...,[<t> `` the late show with david letterman '' ...,<t> `` the late show with david letterman '' c...
3124,( cnn ) feeling so happy you just ca n't stand...,[<t> a new study has found that acetaminophen ...,<t> subjects taking acetaminophen reacted less...
3125,"( cnn ) love it or hate it , jared leto 's int...",[<t> the oscar winner put on white makeup -lrb...,<t> leto will play the clown prince of crime i...
3126,( the hollywood reporter ) the original cast o...,[<t> -lrb- the hollywood reporter -rrb- the or...,<t> `` twin peaks '' creator david lynch annou...


In [28]:
# Only keep xsum data
fib_df = fib_df[fib_df['dataset'] == 'xsum']

In [29]:
fib_df[['input', 'list_choices', 'correct_choice']].head(5)

,input,list_choices,correct_choice
0,Vehicles and pedestrians will now embark and d...,[ A new service on the Isle of Wight's chain f...,Passengers using a chain ferry have been warne...
1,If you leave your mobile phone somewhere do yo...,"[ You may be worried about your health, but wh...","Do you ever feel lonely, stressed or jealous w..."
2,"Speaking on TV, Maria Zakharova said Jews had ...",[ The Russian foreign minister has said she ha...,A spokeswoman on Russian TV has said Jewish pe...
3,"A report by the organisation suggests men, wom...",[ Egyptian police are systematically abusing d...,Egyptian security forces are using sexual viol...
4,Police in Australia and Europe were aware of a...,[One word and a freckle indirectly led to Huck...,One word and a freckle indirectly led to Huckl...


In [30]:
# Each list choice contains a positive and negative summary; we'll explode, clean, and drop duplicates
fib_df = fib_df.explode('list_choices')
fib_df['list_choices'] = fib_df['list_choices'].apply(lambda x: x.strip())
fib_df = fib_df.drop_duplicates(subset=['input', 'list_choices'])
fib_df[['input', 'list_choices', 'correct_choice']].head(5)

,input,list_choices,correct_choice
0,Vehicles and pedestrians will now embark and d...,A new service on the Isle of Wight's chain fer...,Passengers using a chain ferry have been warne...
0,Vehicles and pedestrians will now embark and d...,Passengers using a chain ferry have been warne...,Passengers using a chain ferry have been warne...
1,If you leave your mobile phone somewhere do yo...,"You may be worried about your health, but what...","Do you ever feel lonely, stressed or jealous w..."
1,If you leave your mobile phone somewhere do yo...,"Do you ever feel lonely, stressed or jealous w...","Do you ever feel lonely, stressed or jealous w..."
2,"Speaking on TV, Maria Zakharova said Jews had ...",The Russian foreign minister has said she has ...,A spokeswoman on Russian TV has said Jewish pe...


![](static/nli.png)

We are going to map to 0 and 1 for the sake of it!

In [31]:
# Create labels where factually consistent = 2 (entailment) and factually inconsistent = 0 (contradiction)
# What happened to label = 1? We drop it as it represents neutral in the NLI task
fib_df.loc[fib_df['correct_choice'] == fib_df['list_choices'], 'target'] = 1
fib_df.loc[fib_df['correct_choice'] != fib_df['list_choices'], 'target'] = 0
fib_df['target'] = fib_df['target'].astype(int)

fib_df[['input', 'list_choices', 'correct_choice', 'target']].head()

,input,list_choices,correct_choice,target
0,Vehicles and pedestrians will now embark and d...,A new service on the Isle of Wight's chain fer...,Passengers using a chain ferry have been warne...,0
0,Vehicles and pedestrians will now embark and d...,Passengers using a chain ferry have been warne...,Passengers using a chain ferry have been warne...,1
1,If you leave your mobile phone somewhere do yo...,"You may be worried about your health, but what...","Do you ever feel lonely, stressed or jealous w...",0
1,If you leave your mobile phone somewhere do yo...,"Do you ever feel lonely, stressed or jealous w...","Do you ever feel lonely, stressed or jealous w...",1
2,"Speaking on TV, Maria Zakharova said Jews had ...",The Russian foreign minister has said she has ...,A spokeswoman on Russian TV has said Jewish pe...,0


In [32]:
# Split into train and val, ensuring that the same source doc doesn't appear across train and val
source_grouped = (fib_df.groupby('input')
                  .agg({'target': 'count'})
                  .reset_index())

input_train, input_val = train_test_split(source_grouped,
                                          test_size=0.18, # mistral fine-tune doesn't want more than 20%
                                          stratify=source_grouped['target'],
                                          random_state=1368)

# input_test, input_val = train_test_split(input_val,
#                                          test_size=0.5,
#                                          stratify=input_val['target'],
#                                          random_state=1368)

fib_train = fib_df[fib_df['input'].isin(input_train['input'])]
fib_val = fib_df[fib_df['input'].isin(input_val['input'])]
# fib_test = fib_df[fib_df['input'].isin(input_test['input'])]

In [33]:
# NOTE: In FIB, each doc has 1 positive summary and 5-6 negative summaries. We'll balance it to 1 is to 1.
fib_train = fib_train.drop_duplicates(subset=['input', 'target'])
fib_val = fib_val.drop_duplicates(subset=['input', 'target'])
# fib_test = fib_test.drop_duplicates(subset=['input', 'target'])

In [34]:
fib_train.to_csv('data/fib-train.csv', index=False)
fib_val.to_csv('data/fib-val.csv', index=False)
# fib_test.to_csv('data/fib-test.csv', index=False)

In [35]:
len(fib_train)

820

In [36]:
fib_train.iloc[0]

id                                                           32168497
input               Vehicles and pedestrians will now embark and d...
correct_choice      Passengers using a chain ferry have been warne...
list_choices        A new service on the Isle of Wight's chain fer...
lbl                                                                 1
distractor_model                                            bart-base
dataset                                                          xsum
target                                                              0
Name: 0, dtype: object

In [37]:
# Test loading into dataset
fib_files = {'train': 'data/fib-train.csv',
             'val': 'data/fib-val.csv'}

fib_ds = load_dataset('csv', data_files=fib_files)
fib_ds = fib_ds.select_columns(['input', 'list_choices', 'target'])
fib_ds = fib_ds.rename_column('input', 'premise').rename_column('list_choices', 'hypothesis')

Generating train split: 820 examples [00:00, 23944.92 examples/s]
Generating val split: 180 examples [00:00, 20254.73 examples/s]


In [38]:
fib_ds["train"][5]

{'premise': 'Speaking on TV, Maria Zakharova said Jews had told her they donated both to Mr Trump and Hillary Clinton.\nShe joked that American Jews were the best guide to US politics.\nThe diplomat\'s remarks caused shock. Anti-US propagandists in the last century peddled an idea that rich New York Jews controlled US politics.\nMs Zakharova was speaking on a chat show on Russian state TV at the weekend but her comments drew more attention after being picked up by media outlets on Thursday.\nShe said she had visited New York with an official Russian delegation at the time of the last UN General Assembly, in September.\n"I have a lot of friends and acquaintances there, of course I was interested to find out: how are the elections going, what are the American people\'s expectations?" she said.\n"If you want to know what will happen in America, who do you need to talk to? You have to talk to the Jews, of course. It goes without saying."\nAt this, the TV studio audience applauded loudly.\n

In [39]:
ds_train = fib_ds["train"]
ds_val = fib_ds["val"]
# ds_test = fib_ds["test"]

In [40]:
ds_train.to_json("data/fib-train.jsonl", orient="records", lines=True)
ds_val.to_json("data/fib-val.jsonl", orient="records", lines=True)
# ds_test.to_json("data/fib-test.jsonl", orient="records", lines=True)

Creating json from Arrow format: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 510.19ba/s]


274757

## USB

In [42]:
usb_files = {'train': 'data/usb-train.csv',
             'val': 'data/usb-val.csv'}

usb_ds = load_dataset('csv', data_files=usb_files)
usb_ds = usb_ds.select_columns(['source', 'summary_sent', 'label'])
usb_ds = usb_ds.rename_column('source', 'premise').rename_column('summary_sent', 'hypothesis')
usb_ds = usb_ds.map(lambda x: {"label": 1 if x["label"] == 2 else x["label"]})
usb_ds = usb_ds.rename_column('label', 'target')

usb_ds_train = usb_ds["train"]
usb_ds_val = usb_ds["val"]
usb_ds_train.to_json("data/usb-train.jsonl", orient="records", lines=True)
usb_ds_val.to_json("data/usb-val.jsonl", orient="records", lines=True)

Generating train split: 5050 examples [00:00, 70732.38 examples/s]
Generating val split: 2668 examples [00:00, 108077.02 examples/s]
Creating json from Arrow format: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 204.83ba/s]


1603234

In [43]:
usb_ds_train

Dataset({
    features: ['premise', 'hypothesis', 'target'],
    num_rows: 5050
})